# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## **🗒️ In this notebook we will see how to create a training dataset from the feature groups:** 
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![tutorial-flow](images/02_training-dataset.png) 

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/124
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
trans_fg = fs.get_feature_group('transactions', version=1)
window_aggs_fg = fs.get_feature_group('transactions_4h_aggs', version=1)

In [3]:
# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "category", "amount", "age_at_transaction", "days_until_card_expires", "loc_delta"])\
    .join(window_aggs_fg.select_except(["cc_num"]))

ds_query.show(5)

2022-06-20 09:38:17,044 INFO: USE `robin100_featurestore`
2022-06-20 09:38:18,287 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`fraud_label` `fraud_label`, `fg1`.`category` `category`, `fg1`.`amount` `amount`, `fg1`.`age_at_transaction` `age_at_transaction`, `fg1`.`days_until_card_expires` `days_until_card_expires`, `fg1`.`loc_delta` `loc_delta`, `fg1`.`cc_num` `join_pk_cc_num`, `fg1`.`datetime` `join_evt_datetime`, `fg0`.`trans_volume_mstd` `trans_volume_mstd`, `fg0`.`trans_volume_mavg` `trans_volume_mavg`, `fg0`.`trans_freq` `trans_freq`, `fg0`.`loc_delta_mavg` `loc_delta_mavg`, RANK() OVER (PARTITION BY `fg0`.`cc_num`, `fg1`.`datetime` ORDER BY `fg0`.`datetime` DESC) pit_rank_hopsworks
FROM `robin100_featurestore`.`transactions_1` `fg1`
INNER JOIN `robin100_featurestore`.`transactions_4h_aggs_1` `fg0` ON `fg1`.`cc_num` = `fg0`.`cc_num` AND `fg1`.`datetime` >= `fg0`.`datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`fraud_label` `fraud_label`, `right_fg0`.`ca

,fraud_label,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,Grocery,93.51,25.334094,175.912280,0.000000,93.510,93.510,93.510,0.000000
1,0,Domestic Transport,65.14,25.335632,175.350486,0.319574,65.140,65.140,65.140,0.319574
2,0,Grocery,0.26,25.336235,175.130347,0.314148,0.260,0.260,0.260,0.314148
3,0,Grocery,1.43,25.336660,174.975058,0.000000,0.845,0.845,0.845,0.157074
4,0,Grocery,19.75,25.344710,172.034664,0.105313,19.750,19.750,19.750,0.105313


Recall that we computed the features in `transactions_4h_aggs` using 4-hour aggregates. If we had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join we would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

🤖 Transformation Functions </span>

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [4]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "category": label_encoder,
    "amount": min_max_scaler,
    "trans_volume_mavg": min_max_scaler,
    "trans_volume_mstd": min_max_scaler,
    "trans_freq": min_max_scaler,
    "loc_delta": min_max_scaler,
    "loc_delta_mavg": min_max_scaler,
    "age_at_transaction": min_max_scaler,
    "days_until_card_expires": min_max_scaler,
}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View we may use `fs.create_feature_view()`

In [5]:
feature_view = fs.create_feature_view(
    name='transactions_view',
    query=ds_query,
    labels=["fraud_label"],
    transformation_functions=transformation_functions
)

Feature view created successfully, explore it at https://c.app.hopsworks.ai:443/p/124/fs/72/fv/transactions_view/version/1


## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_train_validation_test_splits()` method.

In [6]:
td_version, td_job = feature_view.create_train_validation_test_splits(
    description = 'transactions_dataset',
    data_format = 'csv',
    val_size = 0.2,
    test_size = 0.1,
    write_options = {'wait_for_job': True},
    coalesce = True,
)

Training dataset job started successfully, you can follow the progress at https://c.app.hopsworks.ai/p/124/jobs/named/transactions_view_1_1_create_fv_td_20062022073937/executions


## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset` or `get_train_validation_test_splits` methods. You need to provide the version of the training dataset you want to retrieve.

In [7]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_view.get_train_validation_test_splits(td_version)

In [8]:
X_train

,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,0.000000,0.010858,0.850452,0.024955,0.000000,0.000000,0.000000,0.027577
1,0,0.000000,0.047378,0.943722,0.035718,0.000000,0.000000,0.000000,0.039471
2,0,0.000000,0.063759,0.132026,0.000044,0.000000,0.000000,0.000000,0.000048
3,0,0.000000,0.340603,0.208466,0.211902,0.000000,0.000000,0.000000,0.234169
4,0,0.000000,0.954661,0.874834,0.183911,0.000000,0.000000,0.000000,0.203237
...,...,...,...,...,...,...,...,...,...
74206,5,0.005703,0.206613,0.228933,0.098071,0.002156,0.002156,0.002156,0.103897
74207,5,0.007304,0.909331,0.728989,0.214688,0.004702,0.004702,0.004702,0.210192
74208,5,0.013629,0.516288,0.314443,0.086154,0.005832,0.005832,0.005832,0.102090
74209,8,0.000170,0.132164,0.816431,0.217638,0.000170,0.000170,0.000170,0.240507


In [9]:
X_val

,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,3.336858e-07,0.835281,0.735671,0.043515,1.052428e-02,1.052428e-02,1.052428e-02,0.088640
1,0,6.673716e-07,0.030266,0.437924,0.207004,6.673716e-07,6.673716e-07,6.673716e-07,0.228756
2,0,6.673716e-07,0.501513,0.770557,0.060860,6.673716e-07,6.673716e-07,6.673716e-07,0.067256
3,0,6.673716e-07,0.668691,0.309189,0.117757,6.673716e-07,6.673716e-07,6.673716e-07,0.130131
4,0,6.673716e-07,0.671936,0.773186,0.189745,6.673716e-07,6.673716e-07,6.673716e-07,0.209683
...,...,...,...,...,...,...,...,...,...
21152,4,1.403816e-03,0.909331,0.728987,0.045351,2.677114e-03,2.677114e-03,2.677114e-03,0.160770
21153,4,1.718482e-03,0.476548,0.233572,0.084373,2.240900e-03,2.240900e-03,2.240900e-03,0.138877
21154,4,2.564709e-03,0.560456,0.835585,0.147691,8.005345e-03,8.005345e-03,8.005345e-03,0.176319
21155,4,3.218733e-03,0.448547,0.643785,0.040216,1.141806e-03,1.141806e-03,1.141806e-03,0.137911


The feature view and training dataset are now visible in the UI

![fg-overview](../images/fv_overview.gif)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook, we will train a model on the dataset we created in this notebook and have quick overview of the lineage.